<a href="https://colab.research.google.com/github/jacmal/CRYPTO-Returns-Strategy-Analysis/blob/main/CRYPTO_Price_Retruns_Trend_Strategy_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import json
import requests
import io

from collections import deque
from datetime import datetime
from scipy.optimize import brute

In [ ]:
# TODO !!!

# DEV PART

# load data sets
url_btc = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BTCUSDT_1h.csv"
url_eth = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_ETHUSDT_1h.csv"
#url_bnb = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BNBUSDT_1h.csv"

site_btc = requests.get(url_btc).content
#site_eth = requests.get(url_eth).content

df_btc = pd.read_csv(io.StringIO(site_btc.decode('utf-8')))
#df_eth = pd.read_csv(io.StringIO(site_eth.decode('utf-8')))

# reverse data sets
df_btc = df_btc.loc[::-1].reset_index(drop=True)
df_btc = df_btc.iloc[1:,:]

#df_eth = df_eth.loc[::-1].reset_index(drop=True)
#df_eth = df_eth.iloc[1:,:]

#
col_to_drop = ['unix', 'symbol', 'Volume USDT', 'tradecount', 'Volume BTC']

df_btc.drop(labels=col_to_drop, axis=1, inplace=True)
#df_eth.drop(labels=col_to_drop, axis=1, inplace=True)

#df_btc.set_index('date', drop=True, inplace=True)
#df_eth.set_index('date', drop=True, inplace=True)

df_btc['Open_btc'] = df_btc['open'].copy()

df_btc.head(2)

In [ ]:
df = df_btc.copy()

In [ ]:
# TODO !!!

PRICE = 'close'
df_f = df_btc[['Date', PRICE]].copy()

# SIMPLE TRADING TECHNICAL INDICATORS
df_f['diff'] = df_f[PRICE] - df_f[PRICE].shift(1)
df_f['diff_%'] = (df_f[PRICE] - df_f[PRICE].shift(1)) / df_f[PRICE].shift(1)
df_f['diff_%_round'] = round((df_f[PRICE] - df_f[PRICE].shift(1)) / df_f[PRICE].shift(1) * 100, 1)
df_f['diff_%_bin'] = round((df_f[PRICE] - df_f[PRICE].shift(1)) / df_f[PRICE].shift(1) * 100)
df_f['returns'] = np.log(df_f[PRICE].div(df_f[PRICE].shift(1)))
df_f['creturns'] = df_f.returns.cumsum().apply(np.exp)
df_f['returns_cummax'] = df_f.returns.cummax().apply(np.exp)
df_f['drow_down'] = df_f.returns_cummax - df_f.returns

In [ ]:
# TODO !!!

################################################################################
# TRADING FEATURES SIGNALS and TRADING STRATEGY ENGINIEERING

# SMA - SIMPLE MOVING AVERAGE
SMA_SHORT = 7
SMA_LONG = 30

df_f['sma_short_'+str(SMA_SHORT)] = df_f[PRICE].rolling(window=SMA_SHORT).mean()
df_f['sma_long_'+str(SMA_LONG)] = df_f[PRICE].rolling(window=SMA_LONG).mean()
    
df_f[f'sma_{SMA_SHORT}-{SMA_LONG}_position'] = np.where(df_f['sma_short_'+str(SMA_SHORT)] > df_f['sma_long_'+str(SMA_LONG)], 1, -1)
df_f[f'sma_{SMA_SHORT}-{SMA_LONG}_strategy'] = df_f[f'sma_{SMA_SHORT}-{SMA_LONG}_position'].shift(1) * df_f.returns
df_f[f'sma_{SMA_SHORT}-{SMA_LONG}_cum_strategy'] = df_f[f'sma_{SMA_SHORT}-{SMA_LONG}_strategy'].cumsum().apply(np.exp)
    
# EMA - EXPONETIAL MOVING AVERAGE
EMA_SHORT = 7
EMA_LONG = 30

df_f['ema_short_'+str(EMA_SHORT)] = df_f[PRICE].ewm(span=EMA_SHORT, peroids=EMA_SHORT).mean()
df_f['ema_long_'+str(EMA_LONG)] = df_f[PRICE].ewm(span=EMA_LONG, peroids=EMA_LONG).mean()
    
df_f[f'positions_ema_{EMA_SHORT}-{EMA_LONG}'] = np.where(df_f['ema_short_'+str(EMA_SHORT)] > df_f['ema_long_'+str(EMA_LONG)], 1, -1)
df_f[f'strategy_ema_{EMA_SHORT}-{EMA_LONG}_strategy'] = df_f[f'positions_ema_{EMA_SHORT}-{EMA_LONG}'].shift(1) * df_f.returns
df_f[f'cum_strategy_ema_{EMA_SHORT}-{EMA_LONG}'] = df_f[f'strategy_ema_{EMA_SHORT}-{EMA_LONG}'].cumsum().apply(np.exp)

# CROSS SMA-EMA
SMA_window = 7
EMA_window = 7

df_f[f'sma-{SMA_window}'] = df_f[PRICE].rolling(window=SMA_window).mean()
df_f[f'ema-{EMA_window}'] = df_f[PRICE].ewm(span=EMA_window, peroids=EMA_window).mean()

df_f[f'positions_sma{SMA_window}-ema{EMA_window}'] = np.where(df_f[f'ema-{EMA_window}'] > df_f[f'sma-{SMA_window}'], 1, -1)
df_f[f'strategy__sma{SMA_window}-ema{EMA_window}'] = df_f[f'positions_sma{SMA_window}-ema{EMA_window}'].shift(1) * df_f.returns
df_f[f'cum_strategy_sma{SMA_window}-ema{EMA_window}'] = df_f[f'strategy__sma{SMA_window}-ema{EMA_window}'].cumsum().apply(np.exp)

# MACD (Moving Average Converegence Diveregence)
EMA_SHORT = 7
EMA_LONG = 30
MACD_SIGNAL_WINDOW = 9

ema_short = df_f[PRICE].ewm(span=EMA_SHORT, peroids=EMA_SHORT).mean()
ema_long = df_f[PRICE].ewm(span=EMA_LONG, peroids=EMA_LONG).mean()
    
df_f[f'MACD_s{EMA_SHORT}-l{EMA_LONG}'] = ema_short - ema_long
df_f[f'MACD_Signal_{EMA_SHORT}-{EMA_LONG}'] = df_f[f'MACD_s{EMA_SHORT}-l{EMA_LONG}'].ewm(span=MACD_SIGNAL_WINDOW, min_periods=MACD_SIGNAL_WINDOW).mean()

df_f[f'positions_MACD_{EMA_SHORT}-{EMA_LONG}'] = np.where((df_f[f'MACD_s{EMA_SHORT}-l{EMA_LONG}'] - df_f[f'MACD_Signal_{EMA_SHORT}-{EMA_LONG}']) > 0, 1, -1)
df_f[f'strategy_MACD_{EMA_SHORT}-{EMA_LONG}'] = df_f['positions_MACD'].shift(1) * df_f.returns
df_f[f'cum_strategy_MACD_{EMA_SHORT}-{EMA_LONG}'] = df_f[f'strategy_MACD_{EMA_SHORT}-{EMA_LONG}'].cumsum().apply(np.exp)

# RSI (Relative Strenght Index)
# RSI > 70 - BUY signal, RSI < 30 SELL signal, 30 <= RSI <= 70 - 
SMA_WINDOW = 20
RSI_LOWER = 30
RSI_UPPER = 70

df_f['UP'] = np.where(df_f.diff > 0, df_diff, 0)
df_f['DOWN'] = np.where(df_f.diff < 0, -df_diff, 0)
df_f['SMA_UP'] = df_f.UP.rolling(SMA_WINDOW).mean()
df_f['SMA_DOWN'] = df_f.DOWN.rolling(SMA_WINDOW).mean()

df_f[f'RSI_w{SMA_WINDOW}'] = df_f.SMA_UP / (df_f.SMA_UP + df_f.SMA_DOWN) * 100

positions_rsi_ones = np.where(df_f[f'RSI_w{SMA_WINDOW}'] > RSI_UPPER, 1, df_f[f'RSI_w{SMA_WINDOW}'])
positions_rsi_m_ones = np.where(positions_rsi_ones < RSI_LOWER, -1, positions_rsi_ones)
df_f[f'positions_RSI_w{SMA_WINDOW}-l{RSI_LOWER}-u{RSI_UPPER}'] = np.whwere((positions_rsi_m_ones != -1) & (positions_rsi_m_ones != 1), 0, positions_rsi_m_ones)
df_f[f'strategy_RSI_w{SMA_WINDOW}-l{RSI_LOWER}-u{RSI_UPPER}'] = df_f[f'positions_RSI_w{SMA_WINDOW}-l{RSI_LOWER}-u{RSI_UPPER}'].shift(1) * df_f.returns
df_f[f'cum_strategy_RSI_w{SMA_WINDOW}-l{RSI_LOWER}-u{RSI_UPPER}'] = df_f[f'strategy_RSI_w{SMA_WINDOW}-l{RSI_LOWER}-u{RSI_UPPER}'].cumsum().apply(np.exp)

# MACD/RSI - TODO

# STOCHASTIC OSCILATOR - TODO

# BOLLINGER BANDS - TODO

# FIBONACCI RETRACEMENT - TODO

################################################################################
df_f.head()

In [ ]:
# TODO !!!

# PREDICTED LABELS

# PREDICT: UP, ZERO, DOWN with TRESHHOLD
ZERO_TRESHHOLD = 0.3
            
actual_value = df_f[PRICE]
next_value = df_f[PRICE].shift(-1)
percentage_value_change = round((next_value - actual_value) / actual_value, 1)

set_ones = np.where(percentage_value_change >= ZERO_TRESHHOLD, 1, percentage_value_change)
set_m_ones = np.where(set_ones <= -ZERO_TRESHHOLD, -1, set_ones)
up_zero_down = np.where((set_m_ones != -1) & (set_m_ones != 1), 0, set_m_ones)
        
df_f[f'y_next_val_{str(ZERO_TRESHHOLD)}'] = up_zero_down

# PREDICT: FUTURE RANGED MEAN VALUE
FUTURE_MEAN_WINDOW = 3

actual_value = df_f[PRICE]
next_value = df_f[PRICE].shift(-FUTURE_MEAN_WINDOW)
percentage_value_change = (next_value - actual_value) / actual_value 
next_mean = percentage_value_change.rolling(window=FUTURE_MEAN_WINDOW).mean()

df_f[f'y_next_mean_{str(window_size)}'] = round(next_mean*100, 1)

################################################################################
df_f.head()

In [ ]:
# TODO !!!

# COMPLETED TRADING STRATEGIES FUNCTIONS FOR HYPERPARAMETERS OPTIMIZATION

# TODO
def sma_strategy(sma, col_name='close')
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    data = df.copy()
    
    data['returns'] = np.log(data[col_name].div(data[col_name].shift(1)))
    data['sma_s'] = data[col_name].rolling(window=int(sma[0])).mean()
    data['sma_l'] = data[col_name].rolling(window=int(sma[1])).mean()
    data.dropna(inplace=True)

    data['positions'] = np.where(data.sma_s > data.sma_l, 1, -1)
    data['strategy'] = data.positions.shift(1) * data.returns
    data.dropna(inplace=True)

    return -data[['returns', 'strategy']].sum().apply(np.exp) # maximize absolute performence

# TODO
def ema_startegy(ema, col_name='close'):
    '''
    Short-Long time window EXPONENTIAL MOVING AVERAGE strategy.

    Parameters:
                   ema (tuple): ("short ema window size", "long ema window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    data = df.copy()

    data['returns'] = np.log(data[col_name].div(data[col_name].shift(1)))
    data['ema_s'] = data[col_name].ewm(span=ema[0], peroids=ema[0]).mean()
    data['ema_l'] = data[col_name].ewm(span=ema[1], peroids=ema[1]).mean()
    data.dropna(inplace=True)

    data['positions'] = np.where(data.ema_s > data.ema_l, 1, -1)
    data['strategy'] = data.positions.shift(1) * data.returns
    data.dropna(inplace=True)

    return -data[['returns', 'strategy']].sum().apply(np.exp) # maximize absolute performence

# TODO
def cross_sma_ema_strategy(sma_ema, col_name='close'):
    '''
    Cross simple moving average - exponential moviong average time window strategy.

    Parameters:
               sma_ema (tuple): ("sma window size", "ema window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    data = df.copy()

    data['returns'] = np.log(data[col_name].div(data[col_name].shift(1)))
    data['sma'] = data[col_name].rolling(window=int(sma_ema[0])).mean()
    data['ema'] = data[col_name].ewm(span=sma_ema[1], peroids=sma_ema[1]).mean()
    data.dropna(inplace=True)

    data['positions'] = np.where(data.ema > data.sma, 1, -1)
    data['strategy'] = data.positions.shift(1) * data.returns
    data.dropna(inplace=True)

    return -data[['returns', 'strategy']].sum().apply(np.exp) # maximize absolute performence

# TODO
def macd_strategy(macd, col_name='close'):
    '''
    MACD time window strategy.

    Parameters:
                  macd (tuple): ("short ema window size", "long ema window size", "signal window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    data = df.copy()

    data['returns'] = np.log(data[col_name].div(data[col_name].shift(1)))
    data['ema_short'] = data[col_name].ewm(span=macd[0], peroids=macd[0]).mean()
    data['ema_long'] = data[col_name].ewm(span=macd[1], peroids=macd[1]).mean()
    data.dropna(inplace=True)

    data['MACD'] = data.ema_short - data.ema_long
    data['MACD_Signal'] = data.MACD.ewm(span=macd[2], min_periods=macd[2]).mean()
    data.dropna(inplace=True)

    data['positions_MACD'] = np.where((data.MACD - data.MACD_Signal) > 0, 1, -1)
    data['strategy_MACD'] = data.positions_MACD.shift(1) * data.returns
    data.dropna(inplace=True)

    return -data[['returns', 'strategy_MACD']].sum().apply(np.exp)

# TODO
def rsi_startegy(rsi, col_name='close'):
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    data = df.copy()

    data['returns'] = np.log(data[col_name].div(data[col_name].shift(1)))
    data['diff'] = data[col_name] - data[col_name].shift(1)
    
    data['UP'] = np.where(data.diff > 0, data.diff, 0)
    data['DOWN'] = np.where(data.diff < 0, -data.diff, 0)
    data['SMA_UP'] = data.UP.rolling(window=rsi[2]).mean()
    data['SMA_DOWN'] = data.DOWN.rolling(window=rsi[2]).mean()
    data['RSI'] = data.SMA_UP / (data.SMA_UP + data.SMA_DOWN) * 100

    positions_rsi_ones = np.where(data.RSI > rsi[1], 1, data.RSI)
    positions_rsi_m_ones = np.where(positions_rsi_ones < rsi[0], -1, positions_rsi_ones)
    data['positions_rsi'] = np.where((positions_rsi_m_ones != -1) & (positions_rsi_m_ones != 1), 0, positions_rsi_m_ones)
    data['strategy_rsi'] = data.positions_rsi.shift(1) * data.returns

    return -data[['returns', 'strategy_rsi']].sum().apply(np.exp)

# TODO
def macd_rsi_strategy():
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    pass

# TODO
def stochastic_oscilator_strategy():
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    pass

# TODO
def bollinger_bands_startegy():
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    pass

# TODO
def fibonacci_retracement():
    '''
    Short-Long time window SIMPLE MOVING AVERAGE strategy.

    Parameters:
                   sma (tuple): ("short sma window size", "long sma window size", 1)
      col_name (str, optional): Price column name. Defaults to "close".  

    Returns:
        OPIS
    '''
    pass

In [ ]:
# TODO !!!

## SEARCHING OPTIMAL HYPERPARAMETERS FOR TRADING STARTEGIES
brute(sma_strategy, ((10, 50, 1), (100, 252, 1)))

In [ ]:
# TODO !!!

# TECHNICAL INDICATORS FUNCTIONS

################################################################################
# SIMPLE TECHNICAL INDICATORS

# TODO
def add_price_diff(col_name='close':
    '''
    Price difference between the previous value and the next value.

    Parameters:
      col_name (str, optional): Price column name. Defaults to "close". 
    
    Returns:
      series_diff (pandas.series): Series with price difference.
    '''
    #df['diff'] = df[col_name] - df[col_name].shift(1)
    series_diff = df[col_name] - df[col_name].shift(1)
    return series_diff

# TODO
def add_diff_percentage(col_name='close'):
    '''
    Percentage price difference between the previous value and the next value.

    Parameters:
      col_name (str, optional): Price column name. Defaults to "close". 
    
    Returns:
      series_diff (pandas.series): Series with percentage price difference.
    '''
    #df['diff_%'] = (df[col_name] - df[col_name].shift(1)) / df[col_name].shift(1)
    df_diff_percentage = (df[col_name] - df[col_name].shift(1)) / df[col_name].shift(1)
    return df_diff_percentage

# TODO
def add_diff_percentage_round(col_name='close', val_round=1):
    df['diff_%_round'] = round((df[col_name] - df[col_name].shift(1)) / df[col_name].shift(1) * 100, val_round)
    
# TODO
def add_diff_percentage_bin(col_name='close'):
    #self.df.dropna(inplace=True)
    df['diff_%_bin'] = round((df[col_name] - df[col_name].shift(1)) / df[col_name].shift(1) * 100)

################################################################################
# FINNACIAL RETURNS FUNCTIONS

# TODO
def add_returns(col_name='close'):
    '''
    Create new columns - logarithimc returns.
    '''
    #self.df.dropna(inplace=True)
    #self.df.reset_index(drop=True, inplace=True)
    df['returns'] = np.log(df[col_name].div(df[col_name].shift(1)))

# TODO
def add_returns_cumsum(col_name='close'):
    returns = np.log(df[col_name].div(df[col_name].shift(1)))    
    df['creturns'] = returns.cumsum().apply(np.exp)   

# TODO
def add_returns_cummax(col_name='close'):
    returns = np.log(df[col_name].div(df[col_name].shift(1)))    
    df['returns_cummax'] = returns.cummax().apply(np.exp)

# TODO
def add_return_drowdown(col_name='close'):
    returns = np.log(df[col_name].div(df[col_name].shift(1)))
    cum_max = returns.cummax().apply(np.exp)    
    df['drow_down'] = cum_max - returns

################################################################################
# MOVING AVERAGES INDICATORS

# TODO
def add_sma(window_size=12, col_name='close'):
    df['sma_'+str(window_size)] = df[col_name].rolling(window=window_size).mean()

# TODO
def add_ema(window_size=12, col_name='close'):
    df['ema'+str(window_size)] = df[col_name].ewm(span=window_size, peroids=window_size).mean()

# TODO
def add_macd(signal_window_size=12, short_ema_window_size=1, long_ema_window_size=2):
    pass

# TODO
def add_rsi(sma_window_size=12, col_name='close'):
    df_temp = df.copy()
    df_temp['diff'] = df_temp[col_name] - df_temp[col_name].shift(1)
    df_temp['UP'] = np.where(df_temp.diff > 0, df_temp.diff, 0)
    df_temp['DOWN'] = np.where(df_temp.diff < 0, -df_temp.diff, 0)
    
    df_temp['SMA_UP'] = df_temp.UP.rolling(window=sma_window_size).mean()
    df_temp['SMA_DOWN'] = df_temp.DOWN.rolling(window=sma_window_size).mean()
    
    df['RSI'] = df_temp.SMA_UP / (df_temp.SMA_UP + df_temp.SMA_DOWN) * 100

################################################################################
# POSITIONS and STARTEGIES

# TODO
def add_short_long_sma_strategy(short_sma_window=7, long_sma_window=30, col_name='close'): # STRATEGY POSITION
    short_sma = df[col_name].rolling(window=short_sma_window).mean()
    long_sma = df[col_name].rolling(window=long_sma_window).mean()
    returns = np.log(df[col_name].div(df[col_name].shift(1)))
    
    position = np.where(short_sma > long_sma, 1, -1)
    strategy = position.shift(1) * returns
    strategy_cumsum = strategy.cumsum().apply(np.exp)
    
    df[f'position_sma_{str(short_sma_windwo)}-{str(long_sma_window)}'] = position
    df[f'strategy_sma_{str(short_sma_windwo)}-{str(long_sma_window)}'] = strategy
    df[f'strategy_sma_cumsum_{str(short_sma_windwo)}-{str(long_sma_window)}'] = strategy_cumsum

# TODO
def add_short_long_ema_strategy(short_ema_window=7, long_ema_window=30, col_name='close'):
    short_ema = df[col_name].ewm(span=short_ema_window, peroids=short_ema_window).mean()
    long_ema = df[col_name].ewm(span=long_ema_window, peroids=long_ema_window).mean()
    returns = np.log(df[col_name].div(df[col_name].shift(1)))
    
    position = np.where(short_sma > long_sma, 1, -1)
    strategy = position.shift(1) * returns
    strategy_cumsum = strategy.cumsum().apply(np.exp)
    
    df[f'position_ema_{str(short_ema_windwo)}-{str(long_ema_window)}'] = position
    df[f'strategy_ema_{str(short_ema_windwo)}-{str(long_ema_window)}'] = strategy
    df[f'cum_strategy_ema_{str(short_ema_windwo)}-{str(long_ema_window)}'] = strategy_cumsum

# TODO
def add_cross_sma_ema_strategy(window_size=12, col_name='close'):
    sma = df[col_name].rolling(window=window_size).mean()
    ema = df[col_name].ewm(span=window_size, peroids=window_size).mean()
    returns = np.log(df[col_name].div(df[col_name].shift(1)))

    position = np.where(ema > sma, 1, -1)
    strategy = position.shift(1) * returns
    strategy_cumsum = strategy.cumsum().apply(np.exp)
    
    df[f'position_cross_smaema_{str(short_ema_windwo)}-{str(long_ema_window)}'] = position
    df[f'strategy_cross_smaema{str(short_ema_windwo)}-{str(long_ema_window)}'] = strategy
    df[f'cum_strategy_cross_smaema_{str(short_ema_windwo)}-{str(long_ema_window)}'] = strategy_cumsum

# TODO
def add_macd_strategy(short_ema_window=7, long_ema_window=30, macd_signal_window=9, col_name='close'):
    short_ema = df[col_name].ewm(span=short_ema_window, peroids=short_ema_window).mean()
    long_ema = df[col_name].ewm(span=long_ema_window, peroids=long_ema_window).mean()
    returns = np.log(df[col_name].div(df[col_name].shift(1)))

    df[f'MACD_s{short_ema_window}-l{long_ema_window}'] = ema_short - ema_long
    df[f'MACD_signal_{macd_signal_window}'] = df[f'MACD_s{short_ema_window}-l{long_ema_window}'].ewm(span=macd_signal_window, min_periods=macd_signal_window).mean()
    df.dropna(inplace=True)

    df[f'positions_MACD_s{short_ema_window}_l{long_ema_window}_w{macd_signal_window}'] = np.where((df[f'MACD_s{short_ema_window}-l{long_ema_window}'] - df[f'MACD_signal_{macd_signal_window}']) > 0, 1, -1)
    df[f'strategy_MACD_s{short_ema_window}_l{long_ema_window}_w{macd_signal_window}'] = df[f'positions_MACD_s{short_ema_window}_l{long_ema_window}_w{macd_signal_window}'].shift(1) * returns
    df[f'cum_strategy_MACD_s{short_ema_window}_l{long_ema_window}_w{macd_signal_window}'] = df[f'strategy_MACD_s{short_ema_window}_l{long_ema_window}_w{macd_signal_window}'].cumsum().apply(np.exp)
    df.dropna(inplace=True)

# TODO
def add_rsi_strategy():
    pass

# TODO
def add_macd_rsi_strategy():
    pass

# TODO
def add_stochastic_oscilator_strategy():
    pass

# TODO
def add_fibonacci_retracement_strategy():
    pass

In [ ]:
# TODO !!!

# APPLY TECHNICAL INDICATORS
def add_all_indicators_train():
    add_diff_percentage()
    add_difference()
    add_diff_percentage()
    add_diff_percentage_round()
    add_diff_percentage_bin()
    add_returns()
    add_returns_cumsum()

    # DEV testing
    # drop first row "NaN" afetr transformation above
    df.dropna(inplace=True)
    # convert to int
    df['diff_%_bin'] = df['diff_%_bin'].astype(int)

    # create a lot of "NaN"
    add_sma()
    add_short_long_sma_positions()

################################################################################
all_indicators()
df.head(3)

In [ ]:
# TODO !!!

# LABELS CREATOR FUNCTIONS

# TODO
def train_labels_creator_UP_ZERO_DOWN(col_name=PRICE_COL_NAME, tresh_hold=0.3):
    actual_value = df[col_name]
    next_value = df[col_name].shift(-1)
    percentage_value_change = round((next_value - actual_value) / actual_value, 1)
        
    set_ones = np.where(percentage_value_change >= tresh_hold, 1, percentage_value_change)
    set_m_ones = np.where(set_ones <= -tresh_hold, -1, set_ones)
    up_zero_down = np.where((set_m_ones != -1) & (set_m_ones != 1), 0, set_m_ones)
        
    df[f'y_next_val_{str(tresh_hold)}'] = up_zero_down

# TODO
def train_labels_creator_RANGED_MEAN(col_name=PRICE_COL_NAME, window_size=7):
    actual_value = df[col_name]
    next_value = df[col_name].shift(-window_size)
    percentage_value_change = (next_value - actual_value) / actual_value 
    next_mean = percentage_value_change.rolling(window=window_size).mean()

    df[f'y_next_mean_{str(window_size)}'] = round(next_mean*100, 1)

# TODO
def add_all_LABELS_TRAIN():
    train_labels_creator_UP_ZERO_DOWN()
    train_labels_creator_RANGED_MEAN()

In [ ]:
# TODO !!!

# PRICE TIME SERIES ANALYSIS

# TODO
def count_range_probability_distribution(window_size=7, min_max=10, col_name='diff_%_bin'):
    lenbin = len(range(-min_max, min_max+1, 1))
    count_dict = {bin: val for bin, val in zip(range(-min_max, min_max+1, 1), np.zeros(lenbin))}
    probs_dict = {bin: val for bin, val in zip(range(-min_max, min_max+1, 1), np.zeros(lenbin))}
    #print(count_dict)

    values = df[col_name].value_counts()
    sum_values = sum(values)

    for idx, val in zip(values.index, values):
        probs = round(val / sum_values, 3)
        if idx in list(count_dict.keys()):
            count_dict[idx] = val
            probs_dict[idx] = probs
            
    #print(idx, val, probs)
    return count_dict, probs_dict

# TODO
def data_hist(col_name='diff_%_bin'):
    df[col_name].hist();